<a href="https://colab.research.google.com/github/MoralesTorres/ML_Notebooks/blob/master/RetoSQLChallengue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sqlite3

# Force correct column names (because the CSV header was not read correctly)
departments = pd.read_csv("departments.csv", header=None, names=["id", "department"])
jobs = pd.read_csv("jobs.csv", header=None, names=["id", "job"])
employees = pd.read_csv(
    "hired_employees.csv",
    header=None,
    names=["id", "name", "datetime", "department_id", "job_id"]
)

# Quick sanity check
print(departments.head())
print(jobs.head())
print(employees.head())

conn = sqlite3.connect("challenge.db")

departments.to_sql("departments", conn, if_exists="replace", index=False)
jobs.to_sql("jobs", conn, if_exists="replace", index=False)
employees.to_sql("employees", conn, if_exists="replace", index=False)

print("✅ Reloaded tables with correct schema!")


   id                department
0   1        Product Management
1   2                     Sales
2   3  Research and Development
3   4      Business Development
4   5               Engineering
   id                        job
0   1        Marketing Assistant
1   2                   VP Sales
2   3         Biostatistician IV
3   4  Account Representative II
4   5               VP Marketing
   id            name              datetime  department_id  job_id
0   1     Harold Vogt  2021-11-07T02:48:42Z            2.0    96.0
1   2        Ty Hofer  2021-05-30T05:43:46Z            8.0     NaN
2   3     Lyman Hadye  2021-09-01T23:27:38Z            5.0    52.0
3   4   Lotti Crowthe  2021-10-01T13:04:21Z           12.0    71.0
4   5  Gretna Lording  2021-10-10T22:22:17Z            6.0    80.0
✅ Reloaded tables with correct schema!


In [2]:
print(pd.read_sql_query("PRAGMA table_info(departments);", conn))
print(pd.read_sql_query("PRAGMA table_info(jobs);", conn))
print(pd.read_sql_query("PRAGMA table_info(employees);", conn))


   cid        name     type  notnull dflt_value  pk
0    0          id  INTEGER        0       None   0
1    1  department     TEXT        0       None   0
   cid name     type  notnull dflt_value  pk
0    0   id  INTEGER        0       None   0
1    1  job     TEXT        0       None   0
   cid           name     type  notnull dflt_value  pk
0    0             id  INTEGER        0       None   0
1    1           name     TEXT        0       None   0
2    2       datetime     TEXT        0       None   0
3    3  department_id     REAL        0       None   0
4    4         job_id     REAL        0       None   0


In [3]:
q1 = """
SELECT
  d.department AS department,
  j.job AS job,
  SUM(CASE WHEN ((CAST(strftime('%m', e.datetime) AS INTEGER)-1)/3 + 1) = 1 THEN 1 ELSE 0 END) AS Q1,
  SUM(CASE WHEN ((CAST(strftime('%m', e.datetime) AS INTEGER)-1)/3 + 1) = 2 THEN 1 ELSE 0 END) AS Q2,
  SUM(CASE WHEN ((CAST(strftime('%m', e.datetime) AS INTEGER)-1)/3 + 1) = 3 THEN 1 ELSE 0 END) AS Q3,
  SUM(CASE WHEN ((CAST(strftime('%m', e.datetime) AS INTEGER)-1)/3 + 1) = 4 THEN 1 ELSE 0 END) AS Q4
FROM employees e
JOIN departments d ON d.id = e.department_id
JOIN jobs j ON j.id = e.job_id
WHERE strftime('%Y', e.datetime) = '2021'
GROUP BY d.department, j.job
ORDER BY d.department ASC, j.job ASC;
"""
df_q1 = pd.read_sql_query(q1, conn)
df_q1.head(20)


,department,job,Q1,Q2,Q3,Q4
0,Accounting,Account Representative IV,1,0,0,0
1,Accounting,Actuary,0,1,0,0
2,Accounting,Analyst Programmer,0,0,1,0
3,Accounting,Budget/Accounting Analyst III,0,1,0,0
4,Accounting,Cost Accountant,0,1,0,0
5,Accounting,Database Administrator III,0,0,0,1
6,Accounting,Desktop Support Technician,0,0,1,0
7,Accounting,Food Chemist,1,0,0,0
8,Accounting,Graphic Designer,0,1,0,0
9,Accounting,Health Coach III,0,0,0,1


In [4]:
q2 = """
WITH dept_counts AS (
  SELECT
    d.id AS id,
    d.department AS department,
    COUNT(*) AS hired
  FROM employees e
  JOIN departments d ON d.id = e.department_id
  WHERE strftime('%Y', e.datetime) = '2021'
  GROUP BY d.id, d.department
)
SELECT
  id,
  department,
  hired
FROM dept_counts
WHERE hired > (SELECT AVG(hired) FROM dept_counts)
ORDER BY hired DESC;
"""
df_q2 = pd.read_sql_query(q2, conn)
df_q2


,id,department,hired
0,8,Support,221
1,5,Engineering,208
2,6,Human Resources,204
3,7,Services,204
4,4,Business Development,187
5,3,Research and Development,151
6,9,Marketing,143


In [5]:
avg_df = pd.read_sql_query("""
WITH dept_counts AS (
  SELECT d.id, d.department, COUNT(*) AS hired
  FROM employees e
  JOIN departments d ON d.id = e.department_id
  WHERE strftime('%Y', e.datetime) = '2021'
  GROUP BY d.id, d.department
)
SELECT AVG(hired) AS avg_hired FROM dept_counts;
""", conn)

avg_df


,avg_hired
0,139.166667


In [6]:
avg_hired = float(avg_df["avg_hired"].iloc[0])
print("avg_hired:", avg_hired)
print("min result hired:", df_q2["hired"].min())
print("all above mean:", (df_q2["hired"] > avg_hired).all())


avg_hired: 139.16666666666666
min result hired: 143
all above mean: True
